In [ ]:
# -*- coding: utf-8 -*-

## "Detection of Accounting Anomalies in the Latent Space using Adversarial Autoencoder Neural Networks"


The content of this lab notebook was jointly developed by Marco Schreyer and Timur Sattarov as part of the 2nd KDD'19 Workshop on "Anomaly Detection in Finance". Further details on the workshop can be found via the workshop <a href="https://sites.google.com/view/kdd-adf-2019">homepage</a>.

Please don't hesitate to contact us in case of any questions via <a href="mailto:marco.schreyer@unisg.ch">marco.schreyer@unisg.ch</a> and <a href="mailto:timur.sattarov@gmail.com">timur.sattarov.timur@gmail.com</a>.

### 1.1 Python Libraries Import

In [ ]:
# importing python utility libraries
import os, sys, random
from datetime import datetime

# importing pytorch libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

# importing data science libraries
import pandas as pd
import random as rd
import numpy as np

# importing python plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')
from IPython.display import Image, display

### 1.2 CUDNN and GPU Verification

To determine if CDNN is available on the server let's execute the cell below to display information about the available CUDNN version:

In [ ]:
# print CUDNN backend version
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] The CUDNN backend version: {}'.format(now, torch.backends.cudnn.version()))

Also, let's display information about the potential GPUs running on the server:

In [ ]:
!nvidia-smi

If CUDNN and GPU's are available let's still specify if we want to use both:

In [ ]:
USE_CUDA = True

### 1.3 Python and PyTorch Verification

Let's execute the cell below to display information about the Python and PyTorch version running on the server:

In [ ]:
# print current Python version
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] The Python version: {}'.format(now, sys.version))

In [ ]:
# print current PyTorch version
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] The PyTorch version: {}'.format(now, torch.__version__))

### 1.4 Random Seed Initialization

Finally, let' set the seeds of random elements in the code e.g. the initialization of the network parameters to guarantee deterministic computation and results:

In [ ]:
# init deterministic seed
seed_value = 1234
rd.seed(seed_value) # set random seed
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU
if (torch.backends.cudnn.version() != None and USE_CUDA == True):
    torch.cuda.manual_seed(seed_value) # set pytorch seed GPU

### 1.5 Folder Structure Initialization

Create notebook structure to store the data as well as the trained models:

In [ ]:
if not os.path.exists('./data'): os.makedirs('./data')  # create data directory
if not os.path.exists('./models'): os.makedirs('./models')  # create trained models directory

## 2. Financial Fraud Detection Dataset

In this section, we will conduct a descriptive analysis of the financial dataset. Furthermore, we will apply some necessary pre-processing steps to train a deep neural network. The dataset is based on a derivation of the **"Synthetic Financial Dataset For Fraud Detection"** by Lopez-Rojas [6] available via the Kaggle predictive modelling and analytics competitions platform that can be obtained using the following link: https://www.kaggle.com/ntnu-testimon/paysim1.

In [ ]:
# load the synthetic ERP dataset
#ori_dataset = pd.read_csv('./data/fraud_dataset_v2.csv')

# load the synthetic ERP dataset
url = 'https://raw.githubusercontent.com/GitiHubi/deepAI/master/data/fraud_dataset_v2.csv'
ori_dataset = pd.read_csv(url)

Let's now inspect the dataset dimensions in terms of the number of journal entries and number of attributes contained:

In [ ]:
# inspect the dataset dimensions
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] Transactional dataset of {} rows and {} columns loaded'.format(now, ori_dataset.shape[0], ori_dataset.shape[1]))

### 2.1 Initial Data and Attribute Assessment

We augmented the dataset and renamed the attributes to appear more similar to a real-world dataset that one usually observes in SAP-ERP systems as part of SAP's Finance and Cost controlling (FICO) module. 

The dataset contains a subset of in total 7 categorical and 2 numerical attributes available in the FICO BKPF (containing the posted journal entry headers) and BSEG (containing the posted journal entry segments) tables. Please, find below a list of the individual attributes as well as a brief description of their respective semantics:

>- `BELNR`: the accounting document number,
>- `BUKRS`: the company code,
>- `BSCHL`: the posting key,
>- `HKONT`: the posted general ledger account,
>- `PRCTR`: the posted profit center,
>- `WAERS`: the currency key,
>- `KTOSL`: the general ledger account key,
>- `DMBTR`: the amount in local currency,
>- `WRBTR`: the amount in document currency.

Let's also have a closer look into the top 10 rows of the dataset:

In [ ]:
# inspect top rows of the ERP dataset
ori_dataset.head(10) 

You may also have noticed the attribute `label` in the data. We will use this field throughout the lab to evaluate the quality of our trained models. The field describes the true nature of each individual transaction of either being a **regular** transaction (denoted by `regular`) or an **anomaly** (denoted by `global` and `local`). Let's have closer look into the distribution of the regular vs. anomalous transactions in the dataset:

In [ ]:
# number of anomalies vs. regular transactions
ori_dataset.label.value_counts()

Ok, the statistic reveals that, similar to real world scenarios, we are facing a highly "unbalanced" dataset. Overall, the dataset contains only a small fraction of **100 (0.018%)** anomalous transactions. While the 100 anomalous entries encompass **70 (0.013%)** "global" anomalies and **30 (0.005%)** "local" anomalies as introduced in section 1.2.

In [ ]:
# remove the "ground-truth" label information for the following steps of the lab
label = ori_dataset.pop('label')

### 2.2 Pre-Processing of Categorical Transaction Attributes

From the initial data assessment above we can observe that the majority of attributes recorded in AIS- and ERP-systems contain categorical (discrete) attribute values, e.g. the posting date, the general-ledger account, the posting type, the currency. Let's visually inspect the distribution of two of the dataset attributes provided, namely (1) the posting key (technically denoted by `BSCHL`) as well as (2) the general ledger account (technically denoted by `HKONT`):

In [ ]:
# prepare to plot posting key and general ledger account side by side
fig, ax = plt.subplots(1,2)
fig.set_figwidth(12)

# plot the distribution of the posting key attribute
g = sns.countplot(x=ori_dataset.loc[label=='regular', 'BSCHL'], ax=ax[0])
g.set_xticklabels(g.get_xticklabels(), rotation=0)
g.set_title('Distribution of BSCHL attribute values')

# plot the distribution of the general ledger account attribute
g = sns.countplot(x=ori_dataset.loc[label=='regular', 'HKONT'], ax=ax[1])
g.set_xticklabels(g.get_xticklabels(), rotation=0)
g.set_title('Distribution of HKONT attribute values');

Unfortunately, neural networks are in general not designed to be trained directly on categorical data and require the attributes to be trained on to be numeric. One simple way to meet this requirement is by applying a technique referred to as "one-hot" encoding. Using this encoding technique, we will derive a numerical representation of each of the categorical attribute values. One-hot encoding creates new binary columns for each categorical attribute value present in the original data.

Let's work through a brief example: The categorical attribute “Receiver” below contains the names "John", "Timur" and "Marco". We "one-hot" encode the names by creating a separate binary column for each possible name value observable in the "Receiver" column. Now, we encode for each transaction that contains the value "John" in the "Receiver" column this observation with 1.0 in the newly created "John" column and 0.0 in all other created name columns.

<img align="middle" style="max-width: 600px; height: auto" src="images/encoding.png">

Using this technique will "one-hot" encode the 6 categorical attributes in the original transactional dataset to obtain a binary ("one-hot" encoded) representation of each attribute. This can be achieved using the get_dummies() function available in the Pandas data science library:

In [ ]:
# select categorical attributes to be "one-hot" encoded
categorical_attr_names = ['KTOSL', 'PRCTR', 'BSCHL', 'HKONT', 'BUKRS', 'WAERS']

# encode categorical attributes into a binary one-hot encoded representation 
ori_dataset_categ_transformed = pd.get_dummies(ori_dataset[categorical_attr_names])

Finally, let's inspect the encoding of 10 sample transactions to see if we have been successfull:

In [ ]:
# inspect encoded sample transactions
ori_dataset_categ_transformed.head(10)

### 2.2 Pre-Processing of Numerical Transaction Attributes

Let's now inspect the distributions of the two numerical attributes contained in the transactional dataset namely, the (1) local currency amount DMBTR and the (2) document currency amount WRBTR:

In [ ]:
# init the plots
fig, ax = plt.subplots(1,2)
fig.set_figwidth(20)

# plot distribution of the local amount attribute
g = sns.distplot(ori_dataset['DMBTR'].tolist(), ax=ax[0])
g.set_title('Distribution of DMBTR amount values')

# set axis-labels 
ax[0].set_xlabel('DMBTR')
ax[0].set_ylabel('density')

# plot distribution of the document amount attribute
g = sns.distplot(ori_dataset['WRBTR'].tolist(), ax=ax[1])
g.set_title('Distribution of WRBTR amount values')

# set axis-labels
ax[1].set_xlabel('WRBTR')
ax[1].set_ylabel('density');

As expected, it can be observed, that for both attributes the distributions of amount values are skewed and encompass a heavy tailed distribution. In order to faster approach a potential global minimum it is good practice to scale and normalize numerical input values prior to network training. Therefore, we first log-scale both variables and second min-max normalize the scaled amounts to the interval [0, 1].

In [ ]:
# select "DMBTR" vs. "WRBTR" attribute
numeric_attr_names = ['DMBTR', 'WRBTR']

# add a small epsilon to eliminate zero values from data for log scaling
numeric_attr = ori_dataset[numeric_attr_names] + 1e-4
numeric_attr = numeric_attr.apply(np.log)

# normalize all numeric attributes to the range [0,1]
ori_dataset_numeric_attr = (numeric_attr - numeric_attr.min()) / (numeric_attr.max() - numeric_attr.min())

Let's now visualize the log-scaled and min-max normalized distributions of both attributes:

In [ ]:
# init the plots
fig, ax = plt.subplots(1,2)
fig.set_figwidth(20)

# plot distribution of the local amount attribute
g = sns.distplot(numeric_attr_vis['DMBTR'].tolist(), ax=ax[0])
g.set_title('Distribution of scaled DMBTR amount values')

# set axis-labels 
ax[0].set_xlabel('DMBTR')
ax[0].set_ylabel('density')

# plot distribution of the local amount attribute
g = sns.distplot(numeric_attr_vis['WRBTR'].tolist(), ax=ax[1])
g.set_title('Distribution of scaled WRBTR amount values')

# set axis-labels
ax[1].set_xlabel('WRBTR')
ax[1].set_ylabel('density');

Ok, let's now visually investigate the scaled distributions of both attributes in terms of the distinct anomaly classes contained in the population of journal entries:

In [ ]:
# append 'label' attribute 
numeric_attr_vis = ori_dataset_numeric_attr.copy()
numeric_attr_vis['label'] = label

# plot the log-scaled and min-max normalized numeric attributes
g = sns.pairplot(data=numeric_attr_vis, vars=numeric_attr_names, hue='label', palette={'regular': 'C0', 'local': 'C3', 'global': 'C1'}, markers=['o', 'x', 'x'])

# set figure title
g.fig.suptitle('Distribution of DMBTR vs. WRBTR amount values', y=1.02)

# set figure size
g.fig.set_size_inches(10, 5)

Ok, as anticipated the numeric attribute values of the "global" anomalies (orange) fall outside the range of the regular amount distributions due to their unusual high amount values. In contrast, the numeric attribute values of the "local" anomalies (red) are much more commingled within the regular transaction amounts.
As DMBTR attribute contains a number of extreme values we might want to visulalize its distribution by omitting those set of extreme values.

### 2.3 Merge Categorical and Numerical Transaction Attributes

Finally, we merge both pre-processed numerical and categorical attributes into a single dataset that we will use for training our deep autoencoder neural network (explained an implemented in the following section 4.):

In [ ]:
# merge categorical and numeric subsets
ori_subset_transformed = pd.concat([ori_dataset_categ_transformed, ori_dataset_numeric_attr], axis = 1)

Now, let's again have a look at the dimensionality of the dataset after we applied the distinct pre-processing steps to the attributes:

In [ ]:
# inspect final dimensions of pre-processed transactional data
ori_subset_transformed.shape

Ok, upon completion of all the pre-processing steps (incl. the exercises) we should end up with an encoded dataset consisting of a total number of 533,009 records (rows) and **618 encoded attributes** (columns). Let's keep the number number of columns in mind since it will define the dimensionality of the input- and output-layer of our deep autoencoder network which we will now implement in the following section.

## 3. Adversarial Autoencoder Neural Network

The Adversarial Autoencoder Neural Network (AAE) architecture, as illustrated in the figure below, extends the concept
of Autoencoder Neural Networks (AE) by imposing an arbitrary prior on the AEs latent space using a GAN training setup. This is achieved by training the AAE jointly in two phases (1) a reconstruction phase as well as (2) an adversarial regularization phase.

In the reconstruction phase, the AAEs encoder network $q_{\theta}(z|x)$ is trained to learn an aggregated posterior distribution $q(z)$ of the journal entries $X$ over the latent code vector $Z$. Thereby, the learned posterior distribution corresponds to a compressed representation of the journal entry characteristics. Similarly to AENs, the decoder
network $p_{\theta}(\hat{x}|z)$ of the AAE utilizes the learned latent code vector representations $Z$ to reconstruct the journal entries $\hat{X}$ as faithfully as possible to minimize the AAEs reconstruction error.

In the regularization phase, an adversarial training setup is applied were the encoder network $q_{\theta}(z|x)$ of the AAE functions as the generator network. In addition, a discriminator network $d_{\theta}(z)$ is attached on top of the learned latent code vector $Z$. Similarly to GANs, the discriminator network of the AAE is trained to distinguish samples of an imposed prior distribution $p(z)$ onto $Z$ from the learned aggregated posterior distribution $q(z)$. In contrast, the encoder network is trained to learn a posterior distribution $p(z) ≈ q(z)$ that fools the discriminator network into thinking that the samples drawn from $q(z)$ originate from the imposed prior distribution $p(z)$.

<img align="middle" style="max-width: 830px; height: auto" src="images/autoencoder_2.png">

### 3.1 AAE Implementation - Encoder / Generator Network $q_{\theta}(z|x)$

Now, let's start implementing an AAE by first implementing the encoder-generator network $q_{\theta}(z|x)$ using PyTorch. For the encoder-generator, we aim to implement a network consisting of **five fully-connected layers**. Furthermore, the encoder-generator is specified by the following number of neurons per layer: "618-256-64-16-4-2". Meaning the first layer consists of 618 neurons (specified by the dimensionality of our input data), the second layer of 256 neurons and the subsequent layers of 64, 16, 4 and 2 neurons respectively.

Some elements of the encoder network code below should be given particular attention:

>- `self.encoder_Lx`: defines the linear transformation of the layer applied to the incoming input: $Wx + b$.
>- `nn.init.xavier_uniform`: inits the layer weights using a uniform distribution according to [9].
>- `nn.init.constant`: inits the layer bias with a constant value of 0.0. 
>- `self.encoder_Rx`: defines the non-linear transformation of the layer: $\sigma(\cdot)$.

We use **"Leaky ReLUs"** as introduced by Xu et al. in [7] to avoid "dying" non-linearities and to speed up training convergence. Leaky ReLUs allow a small gradient even when a particular neuron is not active.

In [ ]:
# define encoder class
class Encoder(nn.Module):

    # define class constructor
    def __init__(self, input_size, hidden_size):

        # call super class constructor
        super(Encoder, self).__init__()

        # specify first layer - in 618, out 256
        self.map_L1 = nn.Linear(input_size, hidden_size[0], bias=True) # init linearity
        nn.init.xavier_uniform_(self.map_L1.weight) # init weights according to [9]
        nn.init.constant_(self.map_L1.bias, 0.0) # constant initialization of the bias
        self.map_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True) # add non-linearity according to [10]

        # specify second layer - in 256, out 64
        self.map_L2 = nn.Linear(hidden_size[0], hidden_size[1], bias=True)
        nn.init.xavier_uniform_(self.map_L2.weight)
        nn.init.constant_(self.map_L2.bias, 0.0)
        self.map_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify third layer - in 64, out 16
        self.map_L3 = nn.Linear(hidden_size[1], hidden_size[2], bias=True)
        nn.init.xavier_uniform_(self.map_L3.weight)
        nn.init.constant_(self.map_L3.bias, 0.0)
        self.map_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify fourth layer - in 16, out 4
        self.map_L4 = nn.Linear(hidden_size[2], hidden_size[3], bias=True)
        nn.init.xavier_uniform_(self.map_L4.weight)
        nn.init.constant_(self.map_L4.bias, 0.0)
        self.map_R4 = torch.nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify fifth layer - in 4, out 2
        self.map_L5 = nn.Linear(hidden_size[3], hidden_size[4], bias=True)
        nn.init.xavier_uniform_(self.map_L5.weight)
        nn.init.constant_(self.map_L5.bias, 0.0)
        self.map_R5 = torch.nn.LeakyReLU(negative_slope=0.4, inplace=True)
        
    # define forward pass
    def forward(self, x):

        # run forward pass through the network
        x = self.map_R1(self.map_L1(x))
        x = self.map_R2(self.map_L2(x))
        x = self.map_R3(self.map_L3(x))
        x = self.map_R4(self.map_L4(x))
        x = self.map_R5(self.map_L5(x))

        # return result
        return x

Now, we are ready to instantiate the encoder-generator model to be trained on the CPU or to be trained on any of the available GPUs (if CUDNN is available and `USE_CUDA` is set to `True`) by execution of the following cell:

In [ ]:
# init training network classes / architectures
encoder_train = Encoder(input_size=ori_subset_transformed.shape[1], hidden_size=[256, 64, 16, 4, 2])

# push to cuda if cudnn is available
if (torch.backends.cudnn.version() != None and USE_CUDA == True):
    encoder_train = encoder_train.cuda()

Once the model is initialized we can visualize the model structure and review the implemented network architecture by execution of the following cell:

In [ ]:
# print the initialized architectures
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] encoder-generator architecture:\n\n{}\n'.format(now, encoder_train))

Looks, great? Excellent!

### 3.2 AAE Implementation - Decoder Network $p_{\theta}(x|z)$

Let's continue the AAE by implementing the corresponding decoder network. The decoder also consists of five fully-connected layers. Furthermore, the decoder network is intended to **symmetrically mirror** the encoder networks architecture by a layer wise inversion "2-4-16-64-256" of the encoder network layers.

In [ ]:
# define decoder class
class Decoder(nn.Module):

    # define class constructor
    def __init__(self, output_size, hidden_size):

        # call super class constructor
        super(Decoder, self).__init__()

        # specify first layer - in 2, out 4
        self.map_L1 = nn.Linear(hidden_size[0], hidden_size[1], bias=True) # init linearity
        nn.init.xavier_uniform_(self.map_L1.weight) # init weights according to [9]
        nn.init.constant_(self.map_L1.bias, 0.0) # constant initialization of the bias
        self.map_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True) # add non-linearity according to [10]

        # specify second layer - in 4, out 16
        self.map_L2 = nn.Linear(hidden_size[1], hidden_size[2], bias=True)
        nn.init.xavier_uniform_(self.map_L2.weight)
        nn.init.constant_(self.map_L2.bias, 0.0)
        self.map_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify third layer - in 16, out 64
        self.map_L3 = nn.Linear(hidden_size[2], hidden_size[3], bias=True)
        nn.init.xavier_uniform_(self.map_L3.weight)
        nn.init.constant_(self.map_L3.bias, 0.0)
        self.map_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify fourth layer - in 64, out 256
        self.map_L4 = nn.Linear(hidden_size[3], hidden_size[4], bias=True)
        nn.init.xavier_uniform_(self.map_L4.weight)
        nn.init.constant_(self.map_L4.bias, 0.0)
        self.map_R4 = nn.LeakyReLU(negative_slope=0.4, inplace=True)
        
        # specify fifth layer - in 256, out 618
        self.map_L5 = nn.Linear(hidden_size[4], output_size, bias=True)
        nn.init.xavier_uniform_(self.map_L5.weight)
        nn.init.constant_(self.map_L5.bias, 0.0)
        self.map_S5 = torch.nn.Sigmoid()

    # define forward pass
    def forward(self, x):

        # run forward pass through the network
        x = self.map_R1(self.map_L1(x))
        x = self.map_R2(self.map_L2(x))
        x = self.map_R3(self.map_L3(x))
        x = self.map_R4(self.map_L4(x))
        x = self.map_S5(self.map_L5(x))

        # return result
        return x

Let's also instantiate the decoder model for CPU or GPU training and convince ourselves that it was successfully initialized by printing and reviewing its architecture by execution of the following cell:

In [ ]:
# init training network classes / architectures
decoder_train = Decoder(output_size=ori_subset_transformed.shape[1], hidden_size=[2, 4, 16, 64, 256])

# push to cuda if cudnn is available
if (torch.backends.cudnn.version() != None) and (USE_CUDA == True):
    decoder_train = decoder_train.cuda()
    
# print the initialized architectures
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] decoder architecture:\n\n{}\n'.format(now, decoder_train))

### 3.3 AAE Implementation - Discriminator Network $d_{\theta}(z)$

Let's, now as a final step, complete the AAE implementation by implementing the discriminator network $d_{\theta}(z)$. The discriminator also consists of five fully-connected layers. Furthermore, the discriminator is specified by the following number of neurons per layer: "256-16-4-2".

In [ ]:
# define discriminator class
class Discriminator(nn.Module):

    # define class constructor
    def __init__(self, input_size, hidden_size, output_size):

        # call super class constructor
        super(Discriminator, self).__init__()

        # specify first layer - in 2, out 256
        self.map_L1 = nn.Linear(input_size, hidden_size[0], bias=True) # init linearity
        nn.init.xavier_uniform_(self.map1_L.weight) # init weights according to [9]
        nn.init.constant_(self.map1_L.bias, 0.0) # constant initialization of the bias
        self.map_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True) # add non-linearity according to [10]

        # specify second layer - in 256, out 16
        self.map_L2 = nn.Linear(hidden_size[0], hidden_size[1], bias=True)
        nn.init.xavier_uniform_(self.map2_L.weight)
        nn.init.constant_(self.map_L2.bias, 0.0)
        self.map_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify third layer - in 16, out 4
        self.map_L3 = nn.Linear(hidden_size[1], hidden_size[2], bias=True)
        nn.init.xavier_uniform_(self.map3_L.weight)
        nn.init.constant_(self.map_L3.bias, 0.0)
        self.map_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)
        
        # specify fourth layer - in 4, out 2
        self.map_L4 = nn.Linear(hidden_size[2], output_size, bias=True)
        nn.init.xavier_uniform_(self.map4_L.weight)
        nn.init.constant_(self.map_L4.bias, 0.0)
        self.map_S4 = torch.nn.Sigmoid()

    # define forward pass
    def forward(self, x):

        # run forward pass through the network
        x = self.map_R1(self.map_L1(x))
        x = self.map_R2(self.map_L2(x))
        x = self.map_R3(self.map_L3(x))
        x = self.map_S4(self.map_L4(x))

        # return result
        return x

Let's also instantiate the discriminator model for CPU or GPU training and convince ourselves that it was successfully initialized by printing and reviewing its architecture by execution of the following cell:

In [ ]:
# init training network classes / architectures
discriminator_train = Discriminator(input_size=2, hidden_size=[256, 16, 4, 2], output_size=1)

# push to cuda if cudnn is available
if (torch.backends.cudnn.version() != None) and (USE_CUDA == True):
    discriminator_train = discriminator_train.cuda()
    
# print the initialized architectures
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] discriminator architecture:\n\n{}\n'.format(now, discriminator_train))

### 3.4 Adversarial Autoencoder Neural Network Training

Now that we have implemented the AAE we are ready to train the network. Prior to starting the training, we need to define an appropriate loss functions, learning rates and parameter optimization techniques. Remember, we aim to train the adversarial autoencoder jointly in two training phases, namely (1) a reconstruction phase as well as (2) a regularization phase. In the following we will set the training parameters of each training phase.

#### 3.4.1 Reconstruction Phase Parameter

In the reconstruction phase, the AAEs encoder network $q_{\theta}(z|x)$ is trained to learn an aggregated posterior distribution $q(z)$ of the journal entries $X$ over the latent code vector $Z$. Thereby, the learned posterior distribution corresponds to a compressed representation of the journal entry characteristics. Similarly to AENs, the decoder network $p_{\theta}(\hat{x}|z)$ of the AAE utilizes the learned latent code vector representations $Z$ to reconstruct the journal entries $\hat{X}$ as faithfully as possible to minimize the AAEs reconstruction error.

To achieve this optimization objective, we calculate (1) the **binary cross-entropy reconstruction error (BCE)** of the categorical attribute value encodings $x^{i}_{cat}$, e.g., the encoded general ledger account ids, and (2) the **mean-squared reconstruction error (MSE)** of the numerical attribute value encodings $x^{i}_{con}$, e.g., the encoded posting amount, as given by:

$\mathcal{L}_{\theta}(x^{i};\hat{x}^{i}) = \gamma \hspace{1mm} \mathcal{L}^{CE}_{\theta}(x^{i}_{cat};\hat{x}^{i}_{cat}) + (1 - \gamma) \hspace{1mm} \mathcal{L}^{MSE}_{\theta}(x^{i}_{con};\hat{x}^{i}_{con})$

for a set of $n$-journal entries $x^{i}$, $i=1,...,n$ and their respective reconstructions $\hat{x}^{i}$ and all journal entry attributes $j=1,...,k$. Luckily, an implementation of the BCE and MSE loss is already available in PyTorch. It can be instantiated "off-the-shelf" via execution of the following PyTorch commands:

In [ ]:
# define the optimization criterion / loss function
reconstruction_criterion_categorical = nn.BCEWithLogitsLoss(reduction='mean')
reconstruction_criterion_numeric = nn.MSELoss(reduction='mean')

# push to cuda if cudnn is available
if (torch.backends.cudnn.version() != None and USE_CUDA == True):
    reconstruction_criterion_categorical = reconstruction_criterion_categorical.cuda()
    reconstruction_criterion_numeric = reconstruction_criterion_numeric.cuda()

We will use the Adam optimization as proposed in [11] and set the learning-rate  l=0.001. Each mini-batch step the optimizer will update the encoder- and decoder-parameters $\theta$ according to degree of reconstruction error.

In [ ]:
# define encoder and decoded learning rate
learning_rate_enc = 1e-3
learning_rate_dec = 1e-3

# define encoder and decoder optimization strategy
encoder_optimizer = optim.Adam(encoder_train.parameters(), lr=learning_rate_enc)
decoder_optimizer = optim.Adam(decoder_train.parameters(), lr=learning_rate_dec)

#### 3.4.2 Regularization Phase Parameter

In the regularization phase, an adversarial training setup is applied were the encoder network $q_{\theta}(z|x)$ of the AAE functions as the generator network. In addition, a discriminator network $d_{\phi}(z)$ is attached on top of the learned latent code vector $Z$. Similarly to GANs, the discriminator network of the AAE is trained to distinguish samples of an imposed prior distribution $p(z)$ onto $Z$ from the learned aggregated posterior distribution $q(z)$. In contrast, the encoder network is trained to learn a posterior distribution $p(z) \approx q(z)$ that fools the discriminator network into thinking that the samples drawn from $q(z)$ originate from the imposed prior distribution $p(z)$.

To achieve this optimization objective, we calculate the **binary cross-entropy reconstruction error (BCE)**, as given by:

In [ ]:
# init the discriminator losses
discriminator_criterion = nn.BCELoss()

# push to cuda if cudnn is available
if (torch.backends.cudnn.version() != None and USE_CUDA == True):
    discriminator_criterion = discriminator_criterion.cuda()

We will use the Adam optimization as proposed in [11] and set the learning-rate  l=0.00001. Each mini-batch step the optimizer will update the generator parameters $\theta$ as well as the discriminator parameters $\phi$ according to degree of discrimination error.

In [ ]:
# define generator and discriminator learning rate
learning_rate_dis_z = 1e-5

# define generator and discriminator optimization strategy
discriminator_optimizer = optim.Adam(encoder_train.parameters(), lr=learning_rate_enc)

Now that we have successfully implemented and defined the three AAE building blocks let's take some time to review the `encoder-generator`, `decoder` and `discriminator` model definition as well as the loss. Please, read the above code and comments carefully and don't hesitate to let us know any questions you might have.

## 4. Creation of the Imposed Latent Prior Distribution $p(z)$

In order to partition the journal entry representations learned by the AAE, we sample from a prior distribution $p(z)$ comprised of a mixture of $\tau$ multivariate isotropic Gaussians $\mathcal{N}(\mu,\mathcal{I})$, where $\mu \in \mathcal{R}^{2}$. In the following example we create a prior distribution $p(z)$ consisting $\tau=5$ isotropic Gaussians.

In [ ]:
# define the number of gaussians
tau = 5 

# define radius of each gaussian
rad = 0.8

# define the sigma of each gaussian
sigma = 0.01

# define the dimensionality of each gaussian
dim = 2

# determine x and y coordinates of the target mixture of gaussians
x_centroid = (radius * np.sin(np.linspace(0, 2 * np.pi, tau, endpoint=False)) + 1) / 2
y_centroid = (radius * np.cos(np.linspace(0, 2 * np.pi, tau, endpoint=False)) + 1) / 2

# determine each gaussians mean (centroid) and standard deviation
mu_gauss = np.vstack([x_centroid, y_centroid]).T

# determine the number of samples to be created per gaussian
samples_per_gaussian = 100000

# iterate over the number of distinct gaussians
for i, mu in enumerate(mu_gauss):

    # case: first gaussian
    if i == 0:

        # randomly sample from gaussion distribution 
        z_continous_samples_all = np.random.normal(mu, sigma, size=(samples_per_gaussian, dim))

    # case: non-first gaussian
    else:

        # randomly sample from gaussian distribution
        z_continous_samples = np.random.normal(mu, sigma, size=(samples_per_gaussian, dim))

        # collect and stack new samples
        z_continous_samples_all = np.vstack([z_continous_samples_all, z_continous_samples])

Let's visually inspect the generated prior distribution $p(z)$:

In [ ]:
# init the plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

# plot reconstruction error scatter plot
ax.scatter(z_continous_samples_all[:, 0], z_continous_samples_all[:, 1], c='C0', marker="o", edgecolors='w', linewidth=0.5) 
ax.set_xlabel('$z_1$')
ax.set_ylabel('$z_2$')

# add plot title
ax.set_title('Prior Latent Space Distribution $p(z)$');

Ok, great. Looks like anticipated.

## 5. Training the Adversarial Autoencoder Neural Network (AAE) Model

In this section, we will train our deep adversarial autoencoder neural network (as implemented in section 3. of the lab) using the encoded transactional data (created in section 2. of the lab) as well as the prior distribution (created in section 4. of the lab). More specifically, we will have a detailed look into the distinct training steps as well as how to monitor the training progress.

### 5.1 Preparing the Network Training Data

Let's now start to train a corresponding model for **100 epochs** and a **mini-batch size of 128** journal entries per batch. This implies that the whole dataset will be fed to the AENN 5 times in chunks of 128 journal entries yielding to 4,165 mini-batches (533,009 journal entries / 128 journal entries per mini-batch) per epoch.

In [ ]:
# specify training parameters
num_epochs = 100
mini_batch_size = 128

During the training phase, we will fetch the individual mini-batches of the entire population of journal entries. To achieve this, we will use PyTorch's `DataLoader` that provides single- or multi-process iterators over a given dataset to load one mini-batch at a time. By enabling `shuffle=True` the data will be reshuffled at every epoch prior to feeding it to the network. 

In [ ]:
# convert pre-processed data to pytorch tensor
torch_dataset = torch.from_numpy(ori_subset_transformed.values).float()

# convert to pytorch tensor - none cuda enabled
dataloader = DataLoader(torch_dataset, batch_size=mini_batch_size, shuffle=True, num_workers=0)
# note: we set num_workers to zero to retrieve deterministic results

# determine if CUDA is available at compute node
if (torch.backends.cudnn.version() != None) and (USE_CUDA == True):
    dataloader = DataLoader(torch_dataset.cuda(), batch_size=mini_batch_size, shuffle=True)

### 4.4 Running the Network Training

Finally, we start training the model. The training procedure of each mini-batch is performed as follows: 

>1. do a forward pass through the encoder-decoder part, 
>2. compute the binary-cross-entropy reconstruction loss $\mathcal{L^{BCE}_{\theta}}(x^{i};\hat{x}^{i})$, 
>3. do a backward pass through the encoder-decoder part, and 
>4. update the parameters of the encoder $f_\theta(\cdot)$ and decoder $g_\theta(\cdot)$ networks.

To ensure learning while training our AENN model we will monitor whether the loss decreases with progressing training. Therefore, we obtain and evaluate the reconstruction performance of the entire dataset after each training epoch. Based on this evaluation we can conclude on the training progress and whether the loss is converging (indicating that the model might not improve any further).

The following elements of the network training code below should be given particular attention:
 
>- `reconstruction_loss.backward()` computes the gradients based on the magnitude of the reconstruction loss,
>- `encoder_optimizer.step()` and `decoder_optimizer.step()` updates the network parameters based on the gradient.

Please also note, that the mini-batch training of the AENN will be executed on the GPU (if CUDNN is available and set accordingly by USE_CUDA=True). However, the evaluation of the reconstruction performance over the entire set of journal entries will be performed at CPU level. Using PyTorch this can be easily achieved with the following commands:

>- `encoder_train.cuda()`: moves all model parameters and buffers to the GPU.
>- `encoder_train.cpu()`: moves all model parameters and buffers to the CPU.

The reason for the switch to the CPU in the evaluation phase is the size of the entire dataset. We aim to compute the reconstruction error over the entire dataset, which in most of the cases does not fit into the GPU's memory.

In addition, after each training epoch we want to save a checkpoint for both the actual `encoder` and `decoder` model. The saved model checkpoints contain a snapshot of the trained model parameter values upon completion of a training epoch. In general, it is good practice, to save checkpoints at regular intervals during training. In case your system crashes during training you are able continue from the last checkpoint rather than start over from scratch.

>- `torch.save()`: saves a checkpoint of the actual encoder and decoder model parameter values to disc.

Initialize the collected training losses: 

In [ ]:
# init collection of training losses
reconstruction_losses = []
discriminator_losses_fake = []
discriminator_losses_real = []
discriminator_losses = []
generator_losses = []

Define the verbose step size of the adversarial autoencoder training process:

In [ ]:
mini_batch_verbose_step = 1000

Let's now start the adversarial autoencoder network training:

In [ ]:
# initialize training adversarial autoencoder model
for epoch in range(num_epochs):

    # init mini batch counter
    mini_batch_count = 0

    # determine if GPU training is enabled
    if (torch.backends.cudnn.version() != None) and (use_cuda == True):

        # set all networks / models in GPU mode
        encoder_train.cuda()
        decoder_train.cuda()
        discriminator_train.cuda()

    # set networks in training mode (apply dropout when needed)
    encoder_train.train()
    decoder_train.train()
    discriminator_train.train()
    
    # start timer
    start_time = datetime.now()

    # iterate over epoch mini batches
    for mini_batch_data in dataloader:

        # increase mini batch counter
        mini_batch_count += 1

        # convert mini batch to torch variable
        mini_batch_torch = torch.FloatTensor(mini_batch_data)
        
        # reset the networks gradients
        encoder_train.zero_grad()
        decoder_train.zero_grad()
        discriminator_train.zero_grad()

        # =================== reconstruction phase =====================
        
        # run autoencoder encoding - decoding
        z_sample = encoder_train(mini_batch_torch)
        mini_batch_reconstruction = decoder_train(z_sample)

        # split input date to numerical and categorical part
        batch_cat = mini_batch_torch[:, :ori_dataset_categ_transformed.shape[1]]
        batch_num = mini_batch_torch[:, ori_dataset_categ_transformed.shape[1]:]
        
        # split reconstruction to numerical and categorical part
        rec_batch_cat = mini_batch_reconstruction[:, :ori_dataset_categ_transformed.shape[1]]
        rec_batch_num = mini_batch_reconstruction[:, ori_dataset_categ_transformed.shape[1]:]

        # backward pass + gradients update
        rec_error_cat = reconstruction_criterion_categorical(input=rec_batch_cat, target=batch_cat)  # one-hot attr error
        rec_error_num = reconstruction_criterion_numeric(input=rec_batch_num, target=batch_num)  # numeric attr error

        # combine both reconstruction errors
        reconstruction_loss = rec_error_cat + rec_error_num
        
        # run backward pass - determine gradients
        reconstruction_loss.backward()
        
        # collect reconstruction loss
        # reconstruction_losses.extend([reconstruction_loss.item()])

        # update network parameter - decoder and encoder
        decoder_optimizer.step()
        encoder_optimizer.step()

        # case: verbose step reached?
        if mini_batch_count % mini_batch_verbose_step == 0:

            # print mini-batch reconstuction results
            now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
            print('[LOG TRAIN {}] epoch: {:04}/{:04}, batch: {:04}, reconstruction loss: {:.4f}'.format(now, epoch + 1, num_epochs, mini_batch_count, reconstruction_loss.item()))

            
        # =================== regularization phase =====================
        # =================== discriminator training ===================

        # set discriminator in evaluation mode
        discriminator_train.eval()

        # generate target latent space data
        z_target_batch = z_continous_samples_all[random.sample(range(0, z_continous_samples_all.shape[0]), mini_batch_size),:]

        # convert to torch tensor
        z_target_batch = torch.FloatTensor(z_target_batch)

        if (torch.backends.cudnn.version() != None) and (use_cuda == True):
            z_target_batch = z_target_batch.cuda()

        # determine mini batch sample generated by the encoder -> fake gaussian sample
        z_fake_gauss = encoder_train(mini_batch_torch)

        # determine discriminator classification of both samples
        d_real_gauss = discriminator_train(z_target_batch) # real sampled gaussian 
        d_fake_gauss = discriminator_train(z_fake_gauss) # fake created gaussian

        # determine discriminator classification target variables
        d_real_gauss_target = torch.FloatTensor(torch.ones(d_real_gauss.shape)) # real -> 1
        d_fake_gauss_target = torch.FloatTensor(torch.zeros(d_fake_gauss.shape)) # fake -> 0

        # determine if GPU training is enabled
        if (torch.backends.cudnn.version() != None) and (use_cuda == True):

            # push tensors to CUDA
            d_real_gauss_target = d_real_gauss_target.cuda()
            d_fake_gauss_target = d_fake_gauss_target.cuda()

        # determine individual discrimination losses
        discriminator_loss_real = discriminator_criterion(target=d_real_gauss_target, input=d_real_gauss) # real loss
        discriminator_loss_fake = discriminator_criterion(target=d_fake_gauss_target, input=d_fake_gauss) # fake loss

        # collection discrimination losses
        discriminator_losses_real.extend([discriminator_loss_real.item()])
        discriminator_losses_fake.extend([discriminator_loss_fake.item()])
        
        # add real loss and fake loss
        discriminator_loss = discriminator_loss_fake + discriminator_loss_real

        # run backward through the discriminator network
        discriminator_loss.backward()

        # update network the discriminator network parameters
        discriminator_optimizer.step()

        # reset the networks gradients
        encoder_train.zero_grad()
        decoder_train.zero_grad()
        discriminator_train.zero_grad()

        # case: verbose step reached?
        if mini_batch_count % mini_batch_verbose_step == 0:

            # print mini-batch discrimination results
            now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
            print('[LOG TRAIN {}] epoch: {:04}/{:04}, batch: {:04}, discriminator loss: {:.4f} [real_{:.4f}/fake_{:.4f}]'.format(now, epoch + 1, num_epochs, mini_batch_count, discriminator_loss.item(), discriminator_loss_real.item(), discriminator_loss_fake.item()))
        
        
        # =================== regularization phase =====================
        # =================== generator training =======================

        # set encoder / generator in training mode
        encoder_train.train()
        
        # reset the encoder / generator networks gradients
        encoder_train.zero_grad()

        # determine fake gaussian sample generated by the encoder / generator
        z_fake_gauss = encoder_train(mini_batch_torch)

        # determine discriminator classification of fake gaussian sample
        d_fake_gauss = discriminator_train(z_fake_gauss)

        # determine discriminator classification target variables
        d_fake_gauss_target = torch.FloatTensor(torch.ones(d_fake_gauss.shape)) # fake -> 1

        # determine if GPU training is enabled
        if (torch.backends.cudnn.version() != None) and (use_cuda == True):

            # push tensors to CUDA
            d_fake_gauss_target = d_fake_gauss_target.cuda()

        # determine discrimination loss of fake gaussian sample
        generator_loss = discriminator_criterion(target=d_fake_gauss_target, input=d_fake_gauss)
        
        # collect generation losses
        # generator_losses.extend([generator_loss.item()])

        # run backward pass - determine gradients
        generator_loss.backward()

        # update network paramaters - encoder / generatorc
        encoder_optimizer.step()

        # reset the networks gradients
        encoder_train.zero_grad()
        decoder_train.zero_grad()
        discriminator_train.zero_grad()

        # case: verbose step reached?
        if mini_batch_count % mini_batch_verbose_step == 0:

            # print mini-batch generation results
            now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
            print('[LOG TRAIN {}] epoch: {:04}/{:04}, batch: {:04}, generator loss: {:.4f}'.format(now, epoch + 1, num_epochs, mini_batch_count, generator_loss.item()))

    # =================== validation on the entire dataset ============================            

    encoder_eval = Encoder(input_size=ori_subset_transformed.shape[1], hidden_size=[256, 64, 16, 4, 2])
    decoder_eval = Decoder(output_size=ori_subset_transformed.shape[1], hidden_size=[2, 4, 16, 64, 256])
    discriminator_eval = Discriminator(input_size=2, hidden_size=[256, 16, 4, 2], output_size=1)

    # convert encoded transactional data to torch Variable
    enc_transactions = torch.FloatTensor(torch_dataset)

    # set networks in evaluation mode
    encoder_eval.eval()
    decoder_eval.eval()
    discriminator_eval.eval()

    # determine latent space representation of all transactions
    z_enc_transactions = encoder_eval(enc_transactions)

    # reconstruct input samples
    reconstruction = decoder_eval(z_enc_transactions)

    # split input transactions into numeric and categorical parts
    input_cat_all = enc_transactions[:, :ori_dataset_categ_transformed.shape[1]]
    input_num_all = enc_transactions[:, ori_dataset_categ_transformed.shape[1]:]
    
    # split reconstruction into numeric and categorical parts
    rec_cat_all = reconstruction[:, :ori_dataset_categ_transformed.shape[1]]
    rec_num_all = reconstruction[:, ori_dataset_categ_transformed.shape[1]:]

    # combine categorical and numerical reconstruction errors
    rec_error_cat_all = reconstruction_criterion_categorical(input=rec_cat_all, target=input_cat_all)
    rec_error_num_all = reconstruction_criterion_numeric(input=rec_num_all, target=input_num_all)

    # combine categorical and numerical reconstruction errors
    rec_error_all = rec_error_cat_all + rec_error_num_all          
    reconstruction_losses.extend([rec_error_all.item()])
    
    # evaluate discriminator error
    z_fake = discriminator_eval(z_enc_transactions)
    
    # determine discriminator classification target variables
    z_fake_target = torch.FloatTensor(torch.ones(z_fake.shape)) # fake -> 1

    # determine discrimination loss of fake gaussian sample
    discriminator_loss = discriminator_criterion(target=z_fake_target, input=z_fake)

    # collect generation losses
    discriminator_losses.extend([discriminator_loss.item()])

    # print reconstuction results
    now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG VALID {}] epoch: [{:04}/{:04}], reconstruction loss: {:.4f}'.format(now, epoch + 1, num_epochs, rec_error_all.item()))
    print('[LOG VALID {}] epoch: [{:04}/{:04}], discriminator loss: {:.4f}'.format(now, epoch + 1, num_epochs, discriminator_loss.item()))

    
    # =================== save model snapshots to disk ============================
    
    # save trained encoder model file to disk
    now = datetime.utcnow().strftime("%Y%m%d-%H_%M_%S")
    encoder_model_name = "{}_ep_{}_encoder_model.pth".format(now, (epoch+1))
    torch.save(encoder_train.state_dict(), os.path.join("./models", encoder_model_name))

    # save trained decoder model file to disk
    decoder_model_name = "{}_ep_{}_decoder_model.pth".format(now, (epoch+1))
    torch.save(decoder_train.state_dict(), os.path.join("./models", decoder_model_name))
    
    # save trained discriminator model file to disk
    decoder_model_name = "{}_ep_{}_discriminator_model.pth".format(now, (epoch+1))
    torch.save(discriminator_train.state_dict(), os.path.join("./models", decoder_model_name))

Let's now evaluate if the loss function is indeed going down with progressing training of the model. Therefore, let's visualize the magnitudes of the losses obtained per training epoch:

In [ ]:
# plot the training progress
plt.plot(range(1, len(reconstruction_losses)+1), reconstruction_losses)

# set plot title
plt.title('AAE training performance')

# set plot axis labels
plt.xlabel('training epochs')
plt.ylabel('reconstruction loss')

In [ ]:
# plot the training progress
plt.plot(range(0, len(discriminator_losses)), discriminator_losses)

# set plot title
plt.title('AENN training performance')

# set plot axis labels
plt.xlabel('training epochs')
plt.ylabel('discrimination loss')

How does the reconstruction loss change as we progress in training our model? After 5 epochs, we can observe that our reconstruction loss already went down significantly and starts to converge nicely. This indicates that our network did a pretty good job in learning the structure and attributes of the journal entries.

But, from the plot we also observe that the model could probably be trained a couple more epochs as the trend of the reconstruction error still decreases for the last few epochs. In order to save time, we will continue the lab using a pre-trained model already trained by 20 epochs.

## 5. Evaluating the Autoencoder Neural Network (AENN) Model

In order, to detect interpretable accounting anomalies in real-world ERP datasets we propose a novel anomaly score utilizing the introduced AAE architecture. The score builds on the regularisation applied throughout the AAE training process, namely the reconstruction error loss and the adversarial loss, described in the following.

In [ ]:
# restore pretrained model checkpoint
encoder_model_name = "20190801-16_43_14_ep_20_encoder_model.pth"
decoder_model_name = "20190801-16_43_14_ep_20_decoder_model.pth"

# init training network classes / architectures
encoder_eval = Encoder(input_size=ori_subset_transformed.shape[1], hidden_size=[256, 64, 16, 4, 2])
decoder_eval = Decoder(output_size=ori_subset_transformed.shape[1], hidden_size=[2, 4, 16, 64, 256])

# load trained models
encoder_eval.load_state_dict(torch.load(os.path.join("models", encoder_model_name)))
decoder_eval.load_state_dict(torch.load(os.path.join("models", decoder_model_name)))

### 5.1 Reconstruction Error (RE)

Journal entries that exhibit anomalous attribute value co-occurrences (local anomalies) tend to result in an increased reconstruction error. This is caused by the compression capability of the AAE architecture. Anomalous and therefore unique attribute co-occurrences exhibit an increased probability of getting lost in the encoders "lossy" compression. As a result, their low dimensional representation will overlap with regular entries in the latent space and are not reconstructed correctly by the decoder. Formally, we obtain the reconstruction error $E$ of each entry $x^i$ and its reconstruction $\hat{x}^i$ as the squared-difference denoted by $E_{\theta^*}^{\tau}(x^{i};\hat{x}^{i}) = \frac{1}{k} \sum_{j=1}^{k}{(x^{i}_{j} - \hat{x}^{i}_{j})}^2$ under optimal model parameters $\theta^*$. Finally, we calculate the normalized reconstruction error $RE$ as expressed by:

\begin{equation}
RE_{\theta^*}^{\tau}(x^{i};\hat{x}^{i}) = \frac{E_{\theta^*}^{\tau}(x^i;\hat{x}^{i}) - E_{\theta^*, min}^{\tau}}{E_{\theta^*, max}^{\tau} - E_{\theta^*, min}^{\tau}},
\end{equation}

where $E_{min}$ and $E_{max}$ denotes the min- and max-values of the obtained reconstruction errors given by $E_{\theta^*}$ and closest mode $\tau$.


In [ ]:
# convert encoded transactional data to torch Variable
enc_transactions = torch.FloatTensor(torch_dataset)

# set networks in evaluation mode (don't apply dropout)
encoder_eval.eval()
decoder_eval.eval()

# determine latent space representation of all transactions
z_enc_transactions = encoder_eval(enc_transactions)

# reconstruct input samples
reconstruction = decoder_eval(z_enc_transactions)

# split input transactions into numeric and categorical parts
input_cat_all = enc_transactions[:, :ori_dataset_categ_transformed.shape[1]]
input_num_all = enc_transactions[:, ori_dataset_categ_transformed.shape[1]:]
# split reconstruction into numeric and categorical parts
rec_cat_all = reconstruction[:, :ori_dataset_categ_transformed.shape[1]]
rec_num_all = reconstruction[:, ori_dataset_categ_transformed.shape[1]:]

# compute rec error
rec_error_cat_all = reconstruction_criterion_categorical_unreduced(input=rec_cat_all, target=input_cat_all).mean(dim=1)
rec_error_num_all = reconstruction_criterion_numeric_unreduced(input=rec_num_all, target=input_num_all).mean(dim=1)

# combine categorical and numerical errors
rec_error_all = rec_error_cat_all + rec_error_num_all

### 5.2 Mode Divergence

Journal entries that exhibit anomalous attribute values (global anomalies) result in an increased divergence from the imposed multi-modal prior, e.g., as in this work the divergence to the modes of an imposed mixture of multivariate isotropic Gaussians $\mathcal{N}(\mu,\mathcal{I})$, where $\mu \in \mathcal{R}^m$ defines the $\tau$ modes of the distinct Gaussians denoted by $\mu =\{\mu^1 \ldots \mu^\tau \}$. Throughout the AAE training, the entries will be "pushed" towards the high probability density regions of the prior by the regularization. In order to be able to discriminate between the imposed prior and the learned aggregated posterior the AAE aims to keep the majority of the entries within the high-density regions (modes) of the prior. In contrast, representations that correspond to rare or anomalous journal entries will tend to differ from the imposed modes and be placed in the priors low-density regions. We use this characteristic and obtain an entry's $x^{i}$ mode divergence $D$ as the Euclidean distance of the entry's learned representation $z^i$ to its closest mode $\mu^\tau$. Formally, we derive the mode divergence as denoted by $D_{\theta^*}^{\tau}(z^{i};\mu) = \min\limits_{\tau} \lVert z^i-\mu^\tau \rVert^2 $ under optimal model parameters $\theta^*$. Finally, we calculate the normalized mode divergence $MD$ as expressed by:

\begin{equation}
MD_{\theta^*}^{\tau}(x^{i}) = \frac{D_{\theta^*}^{\tau}(z^i;\mu) - D_{\theta^*, min}^{\tau}}{D_{\theta^*, max}^{\tau} - D_{\theta^*, min}^{\tau}},
\end{equation}

where $D_{min}$ and $D_{max}$ denotes the min- and max-values of the obtained mode divergences given by $D_{\theta^*}$ and closest mode $\tau$.

In [ ]:
# define the Mode Divergence function according to the formula above
def compute_MD(z, means):
    def compute_euclid_dist(x, y):
        return np.sqrt(np.sum((x - y) ** 2, axis=1))

    distances = np.apply_along_axis(func1d=compute_euclid_dist, axis=1, arr=z, y=means)
    return np.min(distances, axis=1), np.argmin(distances, axis=1)

In [ ]:
# convert latent space representation reconstruction errors to numpy
z_enc = z_enc_transactions.cpu().data.numpy()
rec_error_all = rec_error_all.cpu().data.numpy()

# collect distances to the cluster means in the latent space
cluster_distances, cluster_ids = compute_MD(z=z_enc, means=z_mean)

### scale MD and RE across each cluster modes
# prepare empty arrays of the same shape and dtype
rec_error_all_scaled = np.asarray(rec_error_all)
cluster_distances_scaled = np.asarray(cluster_distances)

# iterate over the cluster modes
for cluster_id in np.unique(cluster_ids).tolist():
    # identify the group of transactions assigned to a cluster
    mask = cluster_ids == cluster_id
    # scale the samples to the range [0,1] across the samples within this cluster
    rec_error_all_scaled[mask] = (rec_error_all[mask] - rec_error_all[mask].min()) / (rec_error_all[mask].ptp())
    cluster_distances_scaled[mask] = (cluster_distances[mask] - cluster_distances[mask].min()) / (cluster_distances[mask].ptp())    
    

### 5.3 Anomaly Score

Quantifying both characteristics for a given journal entry, we can reasonably conclude (1) if the entry is anomalous and (2) if it was created by a "regular" business activity. To detect global and local accounting anomalies in real-world audit scenarios we propose to score each journal entry $x^i$ by its normalized reconstruction error $RE$ regularized and normalized mode divergence $MD$ given by:

\begin{equation}
AS^{\tau}(x^{i};\hat{x}^{i}) = \alpha \times RE_{\theta^*}^{\tau}(x^{i};\hat{x}^{i}) + (1-\alpha) \times MD_{\theta^*}^{\tau}(x^{i}),
\end{equation} 

for each individual journal entry $x^{i}$ and optimal model parameters $\theta^*$ and closest mode $\tau$. We introduce $\alpha$ as a factor to balance both characteristics. 


In [ ]:
# compute Anomaly Score
alpha = 0.8
anomaly_score = alpha * rec_error_all_scaled + (1.0 - alpha) * cluster_distances_scaled

### 5.4 Visualization of the Anomaly Score of Individual Transactions

Once we have collected individual reconstruction errors let's visualize them accordingly:

In [ ]:
# prepare plot
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)

# assign unique id to transactions
plot_data = np.column_stack((np.arange(len(anomaly_score)), anomaly_score))

# obtain regular transactions as well as global and local anomalies
regular_data = plot_data[label == 'regular']
global_outliers = plot_data[label == 'global']
local_outliers = plot_data[label == 'local']

# plot reconstruction error scatter plot
ax.scatter(regular_data[:, 0], regular_data[:, 1], c='C0', marker="o", label='regular', alpha=0.4, edgecolors='C0', linewidth=0.5) # plot regular transactions
ax.scatter(global_outliers[:, 0], global_outliers[:, 1], c='C1', marker="x", label='global', s=60) # plot global outliers
ax.scatter(local_outliers[:, 0], local_outliers[:, 1], c='C3', marker="x", label='local', s=60) # plot local outliers

# add plot legend of transaction classes
ax.legend(loc='best');

### 6.4 Inspection of the Latent Space

In [ ]:
# prepare plot
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)

# obtain regular transactions as well as global and local anomalies
regular_data = z_enc[label == 'regular']
global_outliers = z_enc[label == 'global']
local_outliers = z_enc[label == 'local']

# plot reconstruction error scatter plot
ax.scatter(regular_data[:, 0], regular_data[:, 1], c='C0', marker="o", label='regular', edgecolors='w', linewidth=0.5) # plot regular transactions
ax.scatter(global_outliers[:, 0], global_outliers[:, 1], c='C1', marker="x", label='global', edgecolors='w', s=60) # plot global outliers
ax.scatter(local_outliers[:, 0], local_outliers[:, 1], c='C3', marker="x", label='local', edgecolors='w', s=60) # plot local outliers

# add plot legend of transaction classes
ax.legend(loc='best');

The visualization reveals that the pre-trained model is able to reconstruct the majority of regular journal entries, while failing to do so, for the anomalous ones. As a result, the model reconstruction error can be used to distinguish both "global" anomalies (orange) and "local" anomalies (green) from the regular journal entries (blue).

To further investigate our observation and confirm the initial assumption, let's have a closer look into the journal entries exhibiting a "high" binary cross-entropy reconstruction error >= 0.1. We assume that these journal entries correspond to the "global" anomalies of the original dataset:

In [ ]:
# append labels to original dataset
ori_dataset['label'] = label

# inspect transactions exhibiting a reconstruction error >= 0.2
ori_dataset[anomaly_score >= 0.6]

Let's now also have a closer look into the journal entries exhibiting a "medium" binary cross-entropy reconstruction error >= 0.02 and < 0.1. We assume that these journal entries mostly correspond to the "local" anomalies of the original dataset:

In [ ]:
# inspect transactions exhibiting a anomaly_score < 0.1 and >= 0.05
ori_dataset[(anomaly_score >= 0.5) & (anomaly_score < 0.6)]

In [ ]:
!jupyter nbconvert --to script KDD_2019.ipynb

## References

Major elements of the lab content are inspired by the publication "Detection of Anomalies in Large Scale Accounting Data using Deep Autoencoder Networks", of M. Schreyer, T. Sattarov, D. S. Borth, A. Dengel, and B. Reimer, 2017 (arXiv preprint available under: https://arxiv.org/abs/1709.05254).

[1] ACFE, "Report to the Nations on Occupational Fraud and Abuse", The 2016 Global Fraud Study, Association of Certified Fraud Examiners (ACFE), 2016.

[2] J. T. Wells, "Corporate Fraud Handbook: Prevention and Detection", John Wiley & Sons, 2017.

[3] PwC, "Pulling Fraud Out of the Shadows", The Global Economic Crime and Fraud Survey 2018, PricewaterhouseCoopers LLP, 2018.

[4] S. Markovitch, P. Willmott, "Accelerating the digitization of business processes", McKinsey & Company (2014) 1–5.

[5] SAP, SAP Global Corporate Affairs, Corporate Factsheet 2017, 2017.

[6] E. A. Lopez-Rojas , A. Elmir, and S. Axelsson, "PaySim: A financial mobile money simulator for fraud detection", In: The 28th European Modeling and Simulation Symposium-EMSS, Larnaca, Cyprus, 2016.

[7] G. E. Hinton, and R. R. Salakhutdinov, "Reducing the dimensionality of data with neural networks", science 313, no. 5786: 504-507, 2006.

[8] N. Srivastava, G. E. Hinton, A. Krizhevsky, I. Sutskever, and R. Salakhutdinov, "Dropout: A simple way to prevent neural networks from overfitting", The Journal of Machine Learning Research, 15(1), 1929-1958, 2014.

[9] X. Glorot and Y. Bengio, "Understanding the difficulty of training deep feedforward neural networks", Proceedings of the 13th International Conference on Artificial Intelligence and Statistics (AISTATS), 9:249–256, 2010.

[10] B. Xu, N. Wang, T. Chen, and M. Li, "Empirical Evaluation of Rectified Activations in Convolution Network", ICML Deep Learning Workshop, pages 1–5, 2015.

[11] D. P. Kingma and J. Ba, "Adam: A method for stochastic optimization", International Conference on Learning Representations (ICLR). 2015.

[12] G. E. Hinton, N. Srivastava, A. Krizhevsky, I. Sutskever, R. R. Salakhutdinov. "Improving neural networks by preventing co-adaptation of feature detectors", Technical Report, 2012.

[13] D. P. Kingma, M. Welling. "Auto-encoding variational bayes", arXiv preprint arXiv:1312.6114, 2013.

[14] Makhzani, A., Shlens, J., Jaitly, N., Goodfellow, I., & Frey, B., "Adversarial autoencoders", arXiv preprint arXiv:1511.05644, 2015.
